<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch9_5_pg_579~590.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터세트

- 유명한 영화 데이터 정보 사잍츠인 IMDB의 많은 영화 중 주요 5000개 영화에 대한 메타 정보를 새롭게 가공해 캐글에서 제공하는 데이터 세트
-> 콘텐츠 기반 필터링 수행



# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
- 사용자가 특정 영화를 감상하고, 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성요소 등을 가진 다른 영화를 추천하는 것
- 영화를 선택하느느 데 중요한 요소인 영화 장르 속성을 기반으로 만들것 
- 장르 칼럼 값의 유사도를 비교한 뒤, 그 중 높은 평점을 가지는 영화를 추천하는 방식

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ESAA/2023-1/필사/data/archive/") # working directory를 설정함.

Mounted at /content/drive


# 데이터 로딩 및 가공

In [5]:
movies = pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


- 영화 제목, 개요, 인기도, 평점, 투표수, 예산, 키워드 등 영황에 대한 다영한 메타 정보를 갖고 있음
- 이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출해 새로운 데이터 프레임으로 만들 것

In [7]:
movies_df=movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

- 주의점:  genres, keywords 등의 칼럼은 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기->한번에 여러 값을 표현하기 위한 표기 방식
-> 필요한 정보를 추출할 필요가 있다.

In [9]:
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


In [10]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- 여러 개의 장르 데이터를 갖고 있고, 개별 장르의 명칭은 딕셔너리의 키인 name으로 추출할 수 있음
-> 파이썬 ast 모듈의 literal_eval()함수: 문자열을 을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있음

In [11]:
from ast import literal_eval
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [12]:
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]",6.9,4500,139.082615,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]",6.3,4466,107.376788,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name': 'based on novel'}, {'id': 4289, 'name': 'secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...
3,49026,The Dark Knight Rises,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...",7.6,9106,112.312950,"[{'id': 849, 'name': 'dc comics'}, {'id': 853, 'name': 'crime fighter'}, {'id': 949, 'name': 'te...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c..."
4,49529,John Carter,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...",6.1,2124,43.926995,"[{'id': 818, 'name': 'based on novel'}, {'id': 839, 'name': 'mars'}, {'id': 1456, 'name': 'medal...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster..."


# 장르명만 리스트 객체로 추출
- name 키에 해당하는 값을 찾아 이를 리스트 객체로 변환

In [13]:
movies_df['genres']=movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


# 장르 콘텐츠 유사도 측정
- 장르를 문자열로 변환해 이를 countvector로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하면 쉽게 유사도를 측정할 수 있다. 

1. 문자열로 변환된 genres 칼럼을 count 기반으로 피처 벡터화 변환
2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교 -> 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
3. 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

먼저 genres 칼럼을 문자열로 변환한 뒤 countvectorizer를 이용해 피처 벡터 행렬로 만들기
- 리스트 객체 값으로 구성된 장르 칼럼을 개별 요소를 공백 문자로 구분하는 문자여로 변환 -> 별도의 칼럼으로 저장
: 리스트 객체 내 개별 값을 연속된 문자열로 변환하려면 일반적으로 ('구분문자').join(리스트 객체) 을 사용.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

#countvectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal']=movies_df['genres'].apply(lambda x:(' ').join(x))
movies_df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edg...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",Action Adventure Science Fiction


In [15]:
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


- 4803개 레코드, 276개의 개별 단어 피처로 구성된 피처 벡터 행렬 (장르만 벡터화한 것)
- 사이킷런을 이용해 코사인 유사도를 계산할 것
: 행과 비교 행의 코사인 유사도를 행렬 행태로 반환하는 함수 

Q. 위 벡터화에서 총 276개의 장르가 존재하고 각 이에 대한 벡터를 만들어낸 것인가?

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


- genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬 데이터의 행별 유사도 정보를 갖음
- 결국 movies_df의 행별 장르 유사도 값을 갖고 있는 것
- movies_df를 장르 기준으로 콘텐츠 기반 필터링 수행하려면 movies_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 함 -> 앞에서 생성한 genre_sim을 이용.

- genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스를 추출. 
- 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라 비교 대상 행의 위치 인덱스임을 주의
- ***argsort()[:, ::-1]***: 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값을 간편하게 얻을 수 있음: 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출

In [19]:
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:5])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 [2195 1850 3316 ...  887 2544 4802]
 [ 102 2995   56 ... 3046 3045 2401]]


In [21]:
np.array([[1,2,3],[5,7,3],[8,2,1]]).argsort() # 오름차순 정렬

array([[0, 1, 2],
       [2, 0, 1],
       [2, 1, 0]])

In [22]:
np.array([[1,2,3],[5,7,3],[8,2,1]]).argsort()[:,::-1] # 내림차순 정렬

array([[2, 1, 0],
       [1, 0, 2],
       [0, 1, 2]])

In [27]:
genre_sim[1,129]

0.9999999999999999

In [30]:
movies_df.iloc[1,:]

id                                                                                                                285
title                                                                        Pirates of the Caribbean: At World's End
genres                                                                                   [Adventure, Fantasy, Action]
vote_average                                                                                                      6.9
vote_count                                                                                                       4500
popularity                                                                                                 139.082615
keywords          [ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...
overview          Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...
genres_literal                                          

In [32]:
movies_df.iloc[129,:]

id                                                                                                              10195
title                                                                                                            Thor
genres                                                                                   [Adventure, Fantasy, Action]
vote_average                                                                                                      6.6
vote_count                                                                                                       6525
popularity                                                                                                  86.493424
keywords          [new mexico, banishment, shield, marvel comic, hammer, superhero, based on comic book, redemptio...
overview          Against his father Odin's will, The Mighty Thor - a powerful but arrogant warrior god - reckless...
genres_literal                                          

- 위에서 같은 장르는 무슨 기준으로 정렬을 하는 것일까?

자신인 0번 레코드를 제외하고 가장 유사도가 높은 것이 3494번 레코드라는 의미
Q. 아래는 신기하게 유사도가 엉망진창인데 뭐지.. 위는 0번 레코드에만 시도해본 것인가? -> 아닌듯. 자기 자신에 대한 유사도가 가장 높은 것이 정상 아닌가?

# 장르 콘텐츠 필터링을 이용한 영황 추천
- 장르 유사도에 따라 영화를 추천하는 함수를 생성
- 기반데이터: movies_df 데이터 프레임
- 레코드별 장르 코사인 유사도 인덱스: genre_sim_sorted_ind
- 고객이 선정한 추천 기준이 되는 영화 제목
- 추천할 영화 건수
-> 추천 영화 정보를 가지는 데이터 프레임

In [38]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  # 인자로 입력된 movie_df에서 title 칼럼이 입력된 title_name값인 DF 추출 (해당 영화에 대한 레코드 한 개를 추출하는 것)
  title_movie=df[df['title']==title_name]

  # title_naemd를 가진 df의 index 객체를 ndarray로 반환하고
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top n개의 index 추출
  title_index=title_movie.index.values # (아까 위에서 고른 영화에 대한 인덱스 추출함)
  similar_indexes=sorted_ind[title_index, :(top_n)] # (아까 만든 유사도 행렬에서 위 선택한 영화에 대한 행 부분, 상위 10개 추출)

  # 추출된 top_n index 출력. top_n index는 2차원 데이터임.
  # df에서 index로 사용하기 위해 1차원 array로 변경해야함
  print(similar_indexes)
  similar_indexes=similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

# 영화 '대부'와 장르별로 유사한 영화 10개 추천

In [39]:
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- 추천하기에 어려운 영화가 함께 추천됨
- 뭔가 보완이 필요 -> 영화 평점 정보를 이용

In [40]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


-'vote_average': 영화 평점 정보로, 소수 관객이 특정 영화에만 만점이나 높은 점수를 줘서 왜곡되었을 수 있으므로, 이를 확인하기 위해 sort_values()를 이용해 평점을 오름차순으로 movies_df를 정렬해 10개 출력

- 왜곡된 평점 데이터를 피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요.

- 평가 횟수에 가중치가 부여된 평점 방식을 이용: IMDB

`가중 평점=(v/(v+m)) * R + (m/(v+m)) * C`

> - v : 개별 영화에 평점을 투표한 횟수 = vote_count
> - m : 평점을 부여하기 위한 최소 투표 횟수 = 투표 횟수에 따른 가중치 직접 조절
> - R : 개별 영화에 대한 평균 평점 = vote_average
> - C : 전체 영화에 대한 평균 평점

- m=전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정할 것
- m 값을 높임: 평점 투표 횟수가 많은 영화에 더 많은 가중 평점 부여 

# 상위 60%의 투표 횟수 구하기 

In [41]:
C=movies_df['vote_average'].mean() # 전체 영화에 대한 평균 평점
m=movies_df['vote_count'].quantile(0.6) # count 상위 60%
print('C:', round(C, 3), 'm:', round(m, 3)) 

C: 6.092 m: 370.2


# 기존 평점을 새로운 가장 평점으로 변경하는 함수 생성
- vote_weighted: 새로운 평점 정보 값 만들기


In [43]:
percentile=0.6
m=movies_df['vote_count'].quantile(percentile)
C=movies_df['vote_average'].mean()

def weighted_vote_average(record):
  v=record['vote_count']
  R=record['vote_average']

  return ( (v/(v+m)) * R ) + ( (m/(v+m)) * C )

movies_df['weighted_vote']=movies.apply(weighted_vote_average, axis=1)
movies_df

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",Adventure Fantasy Action,6.838594
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,Action Adventure Crime,6.284091
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",Action Crime Drama Thriller,7.541095
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edg...","John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",Action Adventure Science Fiction,6.098838
...,...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[Action, Crime, Thriller]",6.6,238,14.269792,"[united states–mexico barrier, legs, arms, paper knife, guitar case]","El Mariachi just wants to play his guitar and carry on the family tradition. Unfortunately, the ...",Action Crime Thriller,6.290894
4799,72766,Newlyweds,"[Comedy, Romance]",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by the arrivals of their respective sisters.,Comedy Romance,6.089611
4800,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie]",7.0,6,1.444476,"[date, love at first sight, narration, investigation, team, postal worker]","""Signed, Sealed, Delivered"" introduces a dedicated quartet of civil servants in the Dead Letter ...",Comedy Drama Romance TV Movie,6.106650
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],"When ambitious New York attorney Sam is sent to Shanghai on assignment, he immediately stumbles ...",,6.084894


# 새롭게 부여된 weighted_vote 평점을 기준으로 상위 10개 영화를 추출

In [44]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


# 다시 영화 추천
1. 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정 
2. weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 방식
-> find_sim_movie 함수를 변경

In [45]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie=df[df['title']==title_name]
  title_index=title_movie.index.values

  #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes=sorted_ind[title_index, :(top_n*2)]
  similar_indexes=similar_indexes.reshape(-1)
  
  #기준 영화 인덱스는 제외
  similar_indexes=similar_indexes[similar_indexes != title_index]

  #top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


- 더 나은 영화가 추천되었다.